In [ ]:

    def damage_calculate(   # 伤害计算
            self, 
            attacker_skill: str | None,      # 攻击方技能名（防御回合为None）
            defender_level: str | None,      # 防御方等级（'lv1', 'lv2' 或 None）
            is_countered: bool               # 是否被克制
        ) -> int:

        # 1. 获取基础伤害（防御回合用默认进阶/基础伤害）
        if attacker_skill is None:
            # 防御回合：无玩家攻击，用防御等级反推
            # 默认伤害 = config.json中数值（暂时硬编码，后续可抽离）
            base_damage = 25 if defender_level == "lv2" else 10
        else:
            # 正常回合：从技能元数据读取
            base_damage = self.game.get_skill(attacker_skill).damage
        
        # 2. 判断攻击方技能等级
        is_lv2_attack = False
        if attacker_skill:
            is_lv2_attack = self.game.get_skill(attacker_skill).level == "lv2"
        else:
            # 防御回合：根据防御等级反推
            is_lv2_attack = defender_level == "lv2"
        
        # 3. 防御减免（纯逻辑，数值不再硬编码）
        # lv1防御：减伤50%（基础）或50%（进阶）
        # lv2防御：减伤100%（基础）或80%（进阶）
        reduction = 0
        if defender_level == "lv1":
            reduction = base_damage * 0.5  # 统一50%减伤
        elif defender_level == "lv2":
            reduction = base_damage * (1.0 if not is_lv2_attack else 0.5)  # 基础全减，进阶减50%
        
        # 4. 最终伤害（保底1点）
        damage = max(1, int(base_damage - reduction))
        
        # 5. 克制翻倍
        if is_countered:
            damage *= 2
        
        return damage